In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd
import matplotlib.pyplot as plt
model = read_sbml_model('iML1515.xml')

In [2]:
ncam_e=Metabolite('ncam_e', name='nicotineamide', compartment='e', formula = 'C6H6N2O'  ),
na_c=Metabolite('na_c', name='Nicotinic acid',  compartment='c', formula = 'C6H5NO2' ),
namn_c=Metabolite('namn_c', name='nicotinic acid mononucleotide',  compartment='c', formula = 'C11H14NO9P' ),
nr_c=Metabolite('nr_c', name='nicotineamide riboside',  compartment='c', formula = 'C11H15N2O5' ),
nr_e=Metabolite('nr_e', name='nicotineamide riboside',  compartment='e',formula = 'C11H15N2O5' ),
na_e=Metabolite('na_e', name='Nicotinic acid',  compartment='e', formula = 'C6H5NO2' ),
nh3_c=Metabolite('nh3_c', name='Ammonia',  compartment='c', formula = 'NH3' ),
mets=[ncam_e,na_c,namn_c,nr_c,nr_e,na_e,nh3_c]
for i in range(7):
    model.add_metabolites(mets[i])

In [3]:
# NAM Pathway

# NAM + PRPP <=> NMN + PPi https://www.kegg.jp/entry/2.4.2.12
nico_phos_trans = Reaction('NAMPT')
nico_phos_trans.name = 'nicotinamide-phosphoribosyl transferase'

# NMN = Metabolite (id='nmn_c', compartment = 'c')
# NMN.name = 'Nicotinamide mononucleotide'
# NMN.formula = 'C11H15N2O8P'

nico_phos_trans.add_metabolites (({model.metabolites.prpp_c: -1, 
                                 model.metabolites.ncam_c: -1,
                                 model.metabolites.nmn_c: 1,
                                 model.metabolites.ppi_c: 1}))

#R5P->PRPP https://www.kegg.jp/entry/2.7.6.1
R5P_prpp=Reaction('BaPRS')
R5P_prpp.name= ('Ribose 5 phosphate to PRPP')
R5P_prpp.add_metabolites (({model.metabolites.prpp_c: 1,
                            model.metabolites.amp_c:1,
                            model.metabolites.atp_c:-1,
                                 model.metabolites.r5p_c: -1}))

# Ex_NAM -> NAM https://tcdb.org/search/result.php?tc=9.A.23
nicotine_trans = Reaction('NiaP')
nicotine_trans.name = 'nicotine transporter'

nicotine_trans.add_metabolites (({model.metabolites.ncam_c: 1, 
                                  model.metabolites.ncam_e: -1}))

# Transport reaction cytosol -> extracellular 

nmn_ctoe = Reaction('BMpnuC')
nmn_ctoe.name = 'NMN transporter'
nmn_ctoe.add_metabolites({ model.metabolites.nmn_c: -1,
                          model.metabolites.nmn_e: 1})


In [4]:
# Set up pathway
nam_path = [nico_phos_trans,nicotine_trans,R5P_prpp,nmn_ctoe]
model.add_reactions(nam_path)

In [5]:
model.reactions.NAMPT.lower_bound=-10
model.reactions.BaPRS.lower_bound=-10
model.reactions.NiaP.lower_bound=-10
model.reactions.BMpnuC.lower_bound=-10
model.reactions.NAMPT.reversibility= True
model.reactions.BaPRS.reversibility= True
model.reactions.NiaP.reversibility= True
model.reactions.BMpnuC.reversibility= True

C:\Users\Aljosha\anaconda3\lib\site-packages\cobra\core\reaction.py:810: UserWarning: Setting reaction reversibility is ignored
  warn("Setting reaction reversibility is ignored")


In [13]:
model.reactions.BMpnuC.bounds

(-10, 1000.0)

In [6]:
model.add_boundary(model.metabolites.get_by_id("ncam_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("nr_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("na_e"), type="exchange")
model.reactions.EX_ncam_e.lower_bound=-10
model.reactions.EX_nr_e.lower_bound=-10
model.reactions.EX_na_e.lower_bound=-10
medium=model.medium
medium['EX_ncam_e'] = 10
medium['EX_nr_e'] = 10
medium["EX_na_e"] = 10
medium["EX_glc__D_e"] = 10
model.medium=medium

In [7]:
with model:
    growth = model.optimize()
    max_growth_rate = growth.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
    print('NAM-pathway biomass growth', max_growth_rate,'mmol/gDW*h')

NAM-pathway biomass growth 1.3845699440948724 mmol/gDW*h


In [8]:
with model:
    print('With NAM pathway:')
    model.objective = model.reactions.EX_nmn_e #Setting the objective to the reaction, which is producing NMN
    nmn_production = model.optimize().objective_value
    print('Maximum productivity of NMN =', nmn_production, 'mmol/gDW*h')

    glc_uptake_flux = model.reactions.EX_glc__D_e.flux 
    max_yield = nmn_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-NMN/mmol-glc') 

With NAM pathway:
Maximum productivity of NMN = 10.909090909090908 mmol/gDW*h
Maximum theoretical yield = 1.0909090909090908 mmol-NMN/mmol-glc


In [9]:
# Run double optimization, both biomass and product.
solution=model.optimize()


with model:
    
    # Objective set to NMN
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound=(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])/2
    model.objective = model.reactions.EX_nmn_e
    solution = model.optimize()
    print('Optimized for 1/2 of max. Biomass')
    print('NMN production:', solution.fluxes['EX_nmn_e'],'[mmol gDW^-1 h^-1]')
    print('Biomass growth',solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'],'[mmol gDW^-1 h^-1]')

Max. NMN production: 6.000000000000058 [mmol gDW^-1 h^-1]
Biomass growth 0.6922849720474342 [mmol gDW^-1 h^-1]
